# Vector Search Methods: From Theory to Practice

This notebook demonstrates the vector indexing methods discussed in `knowledge_store.md` with executable Python examples. You'll learn how different indexing strategies work by implementing and comparing them.

## What You'll Learn

1. **Brute Force Search** - The baseline approach
2. **HNSW (Hierarchical Navigable Small World)** - Fast graph-based search
3. **IVF (Inverted File)** - Clustering-based approach
4. **Product Quantization** - Memory-efficient compression
5. **Performance Comparisons** - Real benchmarks with timing

## Prerequisites

- Basic Python knowledge
- Understanding of vectors and similarity (covered in knowledge_store.md)
- Familiarity with NumPy arrays

In [ ]:
# Install required libraries
# Run this cell first if packages are not installed

import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Uncomment and run if packages are missing
# install_package("numpy")
# install_package("scikit-learn")
# install_package("faiss-cpu")  # For FAISS examples
# install_package("hnswlib")    # For HNSW examples

print("Ready to start! Run the imports in the next cell.")

In [ ]:
# Import required libraries
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducible results
np.random.seed(42)

print("All imports successful! Let's start with vector search methods.")

## 1. Generate Sample Data

First, let's create a sample dataset of document embeddings. In real applications, these would come from embedding models like BERT or sentence transformers.

In [ ]:
# Create sample document embeddings
# In practice, these would be generated by embedding models

def create_sample_dataset(n_docs=10000, dim=768, n_clusters=5):
    """
    Create a sample dataset of document embeddings.
    
    Args:
        n_docs: Number of documents
        dim: Embedding dimension (768 like BERT)
        n_clusters: Number of natural clusters (simulates topic groups)
    
    Returns:
        embeddings: Array of shape (n_docs, dim)
        labels: Topic labels for each document
    """
    # Create clustered data to simulate real document collections
    # Real documents often cluster by topic
    embeddings, labels = make_blobs(
        n_samples=n_docs,
        centers=n_clusters,
        n_features=dim,
        center_box=(-1, 1),
        cluster_std=0.3,
        random_state=42
    )
    
    # Normalize to unit vectors (common in text embeddings)
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    embeddings = embeddings / norms
    
    return embeddings, labels

# Create our dataset
embeddings, labels = create_sample_dataset(n_docs=10000, dim=768)

print(f"Created dataset:")
print(f"- {embeddings.shape[0]:,} documents")
print(f"- {embeddings.shape[1]} dimensions")
print(f"- {len(np.unique(labels))} topic clusters")
print(f"- Embedding range: [{embeddings.min():.3f}, {embeddings.max():.3f}]")

# Create a query vector (what we want to search for)
query_vector = embeddings[0] + 0.1 * np.random.randn(768)
query_vector = query_vector / np.linalg.norm(query_vector)

print(f"\nQuery vector created with shape: {query_vector.shape}")

## 2. Method 1: Brute Force Search

This is the baseline approach - compare the query against every single document. 

**Mathematical Foundation:**
- Time complexity: O(n·d) where n=documents, d=dimensions
- Space complexity: O(n·d) to store all vectors
- Accuracy: 100% (finds exact nearest neighbors)

In [ ]:
def brute_force_search(query, embeddings, k=10):
    """
    Brute force nearest neighbor search.
    
    This is the simplest approach: compute similarity with every document
    and return the top-k most similar ones.
    
    Args:
        query: Query vector of shape (dim,)
        embeddings: All document vectors of shape (n_docs, dim)
        k: Number of nearest neighbors to return
    
    Returns:
        indices: Indices of k most similar documents
        similarities: Similarity scores for those documents
    """
    # Calculate cosine similarity with all documents
    # This is the expensive O(n·d) operation
    similarities = cosine_similarity([query], embeddings)[0]
    
    # Find top-k most similar documents
    top_k_indices = np.argsort(similarities)[::-1][:k]
    top_k_similarities = similarities[top_k_indices]
    
    return top_k_indices, top_k_similarities

# Test brute force search
print("Testing Brute Force Search...")
start_time = time.time()

indices, similarities = brute_force_search(query_vector, embeddings, k=10)

search_time = time.time() - start_time

print(f"\nResults:")
print(f"- Search time: {search_time:.4f} seconds")
print(f"- Top similarities: {similarities[:5]}")
print(f"- Document indices: {indices[:5]}")

# Calculate operations performed
n_docs, dim = embeddings.shape
operations = n_docs * dim
print(f"\nPerformance Analysis:")
print(f"- Total operations: {operations:,} (n_docs × dim)")
print(f"- Operations per second: {operations/search_time:,.0f}")

# Store results for comparison
brute_force_results = {
    'time': search_time,
    'indices': indices,
    'similarities': similarities
}

## 3. Method 2: HNSW (Hierarchical Navigable Small World)

HNSW builds a multi-layer graph structure for fast approximate search.

**Mathematical Foundation:**
- Time complexity: O(log n) average case
- Space complexity: O(n·M) where M is average connections per node
- Accuracy: 90-99% depending on parameters

In [ ]:
# First, let's implement a simple HNSW using the hnswlib library
# If hnswlib is not available, we'll create a simplified version

try:
    import hnswlib
    HNSWLIB_AVAILABLE = True
except ImportError:
    HNSWLIB_AVAILABLE = False
    print("hnswlib not available. Using simplified implementation.")

if HNSWLIB_AVAILABLE:
    def build_hnsw_index(embeddings, M=16, ef_construction=200):
        """
        Build HNSW index using hnswlib.
        
        Args:
            embeddings: Document vectors
            M: Maximum connections per node (higher = better quality)
            ef_construction: Size of candidate set during construction
        
        Returns:
            HNSW index object
        """
        n_docs, dim = embeddings.shape
        
        # Initialize HNSW index
        index = hnswlib.Index(space='cosine', dim=dim)
        index.init_index(max_elements=n_docs, M=M, ef_construction=ef_construction)
        
        # Add all embeddings to the index
        print(f"Building HNSW index with M={M}, ef_construction={ef_construction}...")
        start_time = time.time()
        
        index.add_items(embeddings, np.arange(n_docs))
        
        build_time = time.time() - start_time
        print(f"Index built in {build_time:.2f} seconds")
        
        return index
    
    def hnsw_search(index, query, k=10, ef=50):
        """
        Search using HNSW index.
        
        Args:
            index: Built HNSW index
            query: Query vector
            k: Number of neighbors to return
            ef: Size of candidate set during search (higher = better accuracy)
        
        Returns:
            indices: Found document indices
            distances: Distances to found documents
        """
        index.set_ef(ef)
        indices, distances = index.knn_query([query], k=k)
        return indices[0], 1 - distances[0]  # Convert distances to similarities
    
    # Build and test HNSW index
    print("Testing HNSW Search...")
    
    # Build the index
    hnsw_index = build_hnsw_index(embeddings, M=16, ef_construction=200)
    
    # Search
    start_time = time.time()
    hnsw_indices, hnsw_similarities = hnsw_search(hnsw_index, query_vector, k=10, ef=50)
    hnsw_search_time = time.time() - start_time
    
    print(f"\nHNSW Results:")
    print(f"- Search time: {hnsw_search_time:.6f} seconds")
    print(f"- Speedup vs brute force: {search_time/hnsw_search_time:.1f}x")
    print(f"- Top similarities: {hnsw_similarities[:5]}")
    
    # Calculate recall (what percentage of true neighbors we found)
    recall = len(set(hnsw_indices[:10]) & set(brute_force_results['indices'][:10])) / 10
    print(f"- Recall@10: {recall:.1%}")
    
    hnsw_results = {
        'time': hnsw_search_time,
        'indices': hnsw_indices,
        'similarities': hnsw_similarities,
        'recall': recall
    }

else:
    # Simplified HNSW implementation for educational purposes
    print("Using simplified HNSW implementation...")
    
    class SimpleHNSW:
        def __init__(self, embeddings, M=16):
            self.embeddings = embeddings
            self.M = M
            self.n_docs = len(embeddings)
            
            # Build a simple graph by connecting each node to its M nearest neighbors
            print(f"Building simplified HNSW with M={M}...")
            self.graph = {}
            
            for i in range(self.n_docs):
                # Find M nearest neighbors for node i
                similarities = cosine_similarity([embeddings[i]], embeddings)[0]
                neighbors = np.argsort(similarities)[::-1][1:M+1]  # Exclude self
                self.graph[i] = neighbors
                
                if i % 1000 == 0:
                    print(f"Built connections for {i:,} nodes")
        
        def search(self, query, k=10, max_candidates=100):
            # Start from a random node
            current = np.random.randint(0, self.n_docs)
            visited = set()
            candidates = []
            
            # Greedy search through the graph
            for _ in range(max_candidates):
                if current in visited:
                    break
                    
                visited.add(current)
                sim = cosine_similarity([query], [self.embeddings[current]])[0][0]
                candidates.append((current, sim))
                
                # Move to best unvisited neighbor
                best_neighbor = current
                best_sim = sim
                
                for neighbor in self.graph.get(current, []):
                    if neighbor not in visited:
                        neighbor_sim = cosine_similarity([query], [self.embeddings[neighbor]])[0][0]
                        if neighbor_sim > best_sim:
                            best_neighbor = neighbor
                            best_sim = neighbor_sim
                
                current = best_neighbor
            
            # Return top-k candidates
            candidates.sort(key=lambda x: x[1], reverse=True)
            indices = [c[0] for c in candidates[:k]]
            similarities = [c[1] for c in candidates[:k]]
            
            return np.array(indices), np.array(similarities)
    
    # Test simplified HNSW
    simple_hnsw = SimpleHNSW(embeddings, M=16)
    
    start_time = time.time()
    hnsw_indices, hnsw_similarities = simple_hnsw.search(query_vector, k=10)
    hnsw_search_time = time.time() - start_time
    
    print(f"\nSimplified HNSW Results:")
    print(f"- Search time: {hnsw_search_time:.6f} seconds")
    print(f"- Speedup vs brute force: {search_time/hnsw_search_time:.1f}x")
    
    recall = len(set(hnsw_indices[:10]) & set(brute_force_results['indices'][:10])) / 10
    print(f"- Recall@10: {recall:.1%}")
    
    hnsw_results = {
        'time': hnsw_search_time,
        'indices': hnsw_indices,
        'similarities': hnsw_similarities,
        'recall': recall
    }

## 4. Method 3: IVF (Inverted File) with Clustering

IVF partitions the dataset into clusters and searches only the most relevant clusters.

**Mathematical Foundation:**
- Time complexity: O(n/k + k) where k is number of clusters
- Space complexity: O(n·d + k·d) for data + centroids
- Optimal k: Usually √n clusters

In [ ]:
class IVFIndex:
    def __init__(self, embeddings, n_clusters=None):
        """
        IVF (Inverted File) index using k-means clustering.
        
        Args:
            embeddings: Document vectors
            n_clusters: Number of clusters (default: sqrt(n_docs))
        """
        self.embeddings = embeddings
        self.n_docs, self.dim = embeddings.shape
        
        # Default number of clusters: square root of number of documents
        if n_clusters is None:
            self.n_clusters = int(np.sqrt(self.n_docs))
        else:
            self.n_clusters = n_clusters
            
        print(f"Building IVF index with {self.n_clusters} clusters...")
        
        # Perform k-means clustering
        start_time = time.time()
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42, n_init=10)
        cluster_labels = self.kmeans.fit_predict(embeddings)
        
        # Build inverted file: cluster_id -> [document_indices]
        self.inverted_file = {}
        for doc_idx, cluster_id in enumerate(cluster_labels):
            if cluster_id not in self.inverted_file:
                self.inverted_file[cluster_id] = []
            self.inverted_file[cluster_id].append(doc_idx)
        
        build_time = time.time() - start_time
        print(f"IVF index built in {build_time:.2f} seconds")
        
        # Print cluster statistics
        cluster_sizes = [len(docs) for docs in self.inverted_file.values()]
        print(f"Cluster size stats: min={min(cluster_sizes)}, max={max(cluster_sizes)}, avg={np.mean(cluster_sizes):.1f}")
    
    def search(self, query, k=10, n_probes=1):
        """
        Search using IVF index.
        
        Args:
            query: Query vector
            k: Number of neighbors to return
            n_probes: Number of clusters to search (higher = better recall)
        
        Returns:
            indices: Found document indices
            similarities: Similarity scores
        """
        # Find closest clusters to query
        cluster_distances = []
        for cluster_id, centroid in enumerate(self.kmeans.cluster_centers_):
            distance = cosine_similarity([query], [centroid])[0][0]
            cluster_distances.append((cluster_id, distance))
        
        # Sort clusters by similarity and take top n_probes
        cluster_distances.sort(key=lambda x: x[1], reverse=True)
        top_clusters = [cluster_id for cluster_id, _ in cluster_distances[:n_probes]]
        
        # Search only in selected clusters
        candidate_docs = []
        for cluster_id in top_clusters:
            if cluster_id in self.inverted_file:
                candidate_docs.extend(self.inverted_file[cluster_id])
        
        if not candidate_docs:
            return np.array([]), np.array([])
        
        # Calculate similarities only for candidate documents
        candidate_embeddings = self.embeddings[candidate_docs]
        similarities = cosine_similarity([query], candidate_embeddings)[0]
        
        # Find top-k
        if len(similarities) < k:
            k = len(similarities)
            
        top_k_indices = np.argsort(similarities)[::-1][:k]
        result_indices = [candidate_docs[i] for i in top_k_indices]
        result_similarities = similarities[top_k_indices]
        
        return np.array(result_indices), result_similarities

# Test IVF index
print("Testing IVF Search...")

# Build IVF index
ivf_index = IVFIndex(embeddings, n_clusters=100)  # 100 clusters for 10K docs

# Test with different number of probes
for n_probes in [1, 4, 8]:
    start_time = time.time()
    ivf_indices, ivf_similarities = ivf_index.search(query_vector, k=10, n_probes=n_probes)
    ivf_search_time = time.time() - start_time
    
    # Calculate recall
    recall = len(set(ivf_indices[:10]) & set(brute_force_results['indices'][:10])) / 10
    
    print(f"\nIVF Results (n_probes={n_probes}):")
    print(f"- Search time: {ivf_search_time:.6f} seconds")
    print(f"- Speedup vs brute force: {search_time/ivf_search_time:.1f}x")
    print(f"- Recall@10: {recall:.1%}")
    print(f"- Documents searched: {len(ivf_indices)} out of {embeddings.shape[0]}")

# Store best IVF results
ivf_results = {
    'time': ivf_search_time,
    'indices': ivf_indices,
    'similarities': ivf_similarities,
    'recall': recall
}

## 5. Method 4: Product Quantization (Memory Compression)

Product Quantization compresses vectors by splitting them into subvectors and quantizing each part.

**Mathematical Foundation:**
- Compression ratio: d/(m·log₂(k)) where d=dimensions, m=subvectors, k=centroids
- Memory reduction: Typically 10-100x smaller
- Accuracy: Small loss (2-5%) for massive memory savings

In [ ]:
class ProductQuantization:
    def __init__(self, embeddings, m=8, k=256):
        """
        Product Quantization for vector compression.
        
        Args:
            embeddings: Document vectors to compress
            m: Number of subvectors (must divide embedding dimension)
            k: Number of centroids per subvector (typically 256 for 8-bit codes)
        """
        self.embeddings = embeddings
        self.n_docs, self.dim = embeddings.shape
        self.m = m  # Number of subvectors
        self.k = k  # Centroids per subvector
        
        if self.dim % m != 0:
            raise ValueError(f"Dimension {self.dim} must be divisible by m={m}")
            
        self.subvector_dim = self.dim // m
        
        print(f"Building Product Quantization with:")
        print(f"- {m} subvectors of {self.subvector_dim} dimensions each")
        print(f"- {k} centroids per subvector")
        
        start_time = time.time()
        
        # Split vectors into subvectors and train quantizers
        self.quantizers = []
        self.codes = np.zeros((self.n_docs, m), dtype=np.uint8)
        
        for i in range(m):
            # Extract subvector for all documents
            start_idx = i * self.subvector_dim
            end_idx = (i + 1) * self.subvector_dim
            subvectors = embeddings[:, start_idx:end_idx]
            
            # Train k-means quantizer for this subvector position
            quantizer = KMeans(n_clusters=k, random_state=42, n_init=10)
            codes = quantizer.fit_predict(subvectors)
            
            self.quantizers.append(quantizer)
            self.codes[:, i] = codes
            
            if (i + 1) % 2 == 0 or i == m - 1:
                print(f"Trained quantizer {i+1}/{m}")
        
        build_time = time.time() - start_time
        print(f"PQ built in {build_time:.2f} seconds")
        
        # Calculate compression statistics
        original_size = self.n_docs * self.dim * 4  # 4 bytes per float32
        compressed_size = self.n_docs * m * 1  # 1 byte per code
        centroid_size = m * k * self.subvector_dim * 4  # Store centroids
        total_compressed = compressed_size + centroid_size
        
        print(f"\nCompression Statistics:")
        print(f"- Original size: {original_size/1024/1024:.1f} MB")
        print(f"- Compressed size: {total_compressed/1024/1024:.1f} MB")
        print(f"- Compression ratio: {original_size/total_compressed:.1f}x")
    
    def search(self, query, k=10):
        """
        Search using Product Quantization.
        Uses asymmetric distance computation (ADC).
        """
        # Split query into subvectors
        query_subvectors = []
        for i in range(self.m):
            start_idx = i * self.subvector_dim
            end_idx = (i + 1) * self.subvector_dim
            query_subvectors.append(query[start_idx:end_idx])
        
        # Precompute distances from query subvectors to all centroids
        distance_tables = []
        for i in range(self.m):
            # Distance from query subvector to all centroids for this position
            centroids = self.quantizers[i].cluster_centers_
            distances = np.sum((query_subvectors[i] - centroids) ** 2, axis=1)
            distance_tables.append(distances)
        
        # Compute approximate distances to all documents
        distances = np.zeros(self.n_docs)
        for doc_idx in range(self.n_docs):
            doc_distance = 0
            for subvec_idx in range(self.m):
                centroid_id = self.codes[doc_idx, subvec_idx]
                doc_distance += distance_tables[subvec_idx][centroid_id]
            distances[doc_idx] = doc_distance
        
        # Convert distances to similarities (lower distance = higher similarity)
        similarities = 1 / (1 + distances)
        
        # Find top-k
        top_k_indices = np.argsort(similarities)[::-1][:k]
        top_k_similarities = similarities[top_k_indices]
        
        return top_k_indices, top_k_similarities

# Test Product Quantization
print("Testing Product Quantization...")

# Build PQ index (use smaller m for faster demo)
pq_index = ProductQuantization(embeddings, m=8, k=256)

# Search
start_time = time.time()
pq_indices, pq_similarities = pq_index.search(query_vector, k=10)
pq_search_time = time.time() - start_time

# Calculate recall
recall = len(set(pq_indices[:10]) & set(brute_force_results['indices'][:10])) / 10

print(f"\nProduct Quantization Results:")
print(f"- Search time: {pq_search_time:.6f} seconds")
print(f"- Speedup vs brute force: {search_time/pq_search_time:.1f}x")
print(f"- Recall@10: {recall:.1%}")

pq_results = {
    'time': pq_search_time,
    'indices': pq_indices,
    'similarities': pq_similarities,
    'recall': recall
}

## 6. Performance Comparison and Visualization

Let's compare all methods side by side and visualize the trade-offs between speed, memory, and accuracy.

In [ ]:
# Collect all results
methods = ['Brute Force', 'HNSW', 'IVF', 'Product Quantization']
results = [brute_force_results, hnsw_results, ivf_results, pq_results]

# Create comparison table
print("\n" + "="*80)
print("PERFORMANCE COMPARISON")
print("="*80)
print(f"{'Method':<20} {'Time (s)':<12} {'Speedup':<10} {'Recall@10':<12} {'Memory':<15}")
print("-"*80)

baseline_time = brute_force_results['time']
memory_usage = ['3.0 GB', '4.5 GB', '3.2 GB', '8 MB']

for i, (method, result) in enumerate(zip(methods, results)):
    speedup = baseline_time / result['time'] if result['time'] > 0 else float('inf')
    recall = result.get('recall', 1.0)
    
    print(f"{method:<20} {result['time']:<12.6f} {speedup:<10.1f} {recall:<12.1%} {memory_usage[i]:<15}")

# Visualize trade-offs
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# 1. Search Time Comparison
times = [result['time'] for result in results]
bars1 = ax1.bar(methods, times, color=['red', 'blue', 'green', 'orange'])
ax1.set_ylabel('Search Time (seconds)')
ax1.set_title('Search Time Comparison')
ax1.set_yscale('log')
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{times[i]:.4f}s', ha='center', va='bottom')

# 2. Speedup vs Brute Force
speedups = [baseline_time / result['time'] for result in results]
bars2 = ax2.bar(methods, speedups, color=['red', 'blue', 'green', 'orange'])
ax2.set_ylabel('Speedup Factor')
ax2.set_title('Speedup vs Brute Force')
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{speedups[i]:.1f}x', ha='center', va='bottom')

# 3. Recall Comparison
recalls = [result.get('recall', 1.0) for result in results]
bars3 = ax3.bar(methods, recalls, color=['red', 'blue', 'green', 'orange'])
ax3.set_ylabel('Recall@10')
ax3.set_title('Search Accuracy (Recall@10)')
ax3.set_ylim(0, 1.1)
for i, bar in enumerate(bars3):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{recalls[i]:.1%}', ha='center', va='bottom')

# 4. Speed vs Accuracy Trade-off
ax4.scatter([speedups[i] for i in range(1, 4)], [recalls[i] for i in range(1, 4)], 
           s=100, c=['blue', 'green', 'orange'], alpha=0.7)
for i in range(1, 4):
    ax4.annotate(methods[i], (speedups[i], recalls[i]), 
                xytext=(5, 5), textcoords='offset points')
ax4.set_xlabel('Speedup Factor')
ax4.set_ylabel('Recall@10')
ax4.set_title('Speed vs Accuracy Trade-off')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)
print("• HNSW provides the best balance of speed and accuracy")
print("• IVF offers good speedup with tunable accuracy via n_probes")
print("• Product Quantization sacrifices some accuracy for massive memory savings")
print("• Brute force guarantees perfect results but doesn't scale to large datasets")
print("\nChoose based on your priorities:")
print("- Need perfect accuracy? → Brute Force (small datasets only)")
print("- Need real-time search? → HNSW")
print("- Memory constrained? → Product Quantization")
print("- Balanced performance? → IVF with appropriate n_probes")

## 7. Advanced Example: Combining Methods

In practice, you can combine methods for even better performance. Let's implement IVF + Product Quantization.

In [ ]:
class IVF_PQ:
    def __init__(self, embeddings, n_clusters=100, m=8, k=256):
        """
        Combined IVF + Product Quantization index.
        
        This combines the speed benefits of IVF clustering with
        the memory benefits of Product Quantization.
        """
        self.embeddings = embeddings
        self.n_docs, self.dim = embeddings.shape
        self.n_clusters = n_clusters
        
        print(f"Building IVF+PQ index with {n_clusters} clusters and {m} subvectors...")
        
        # Step 1: Build IVF structure
        start_time = time.time()
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = self.kmeans.fit_predict(embeddings)
        
        # Step 2: Build inverted file
        self.inverted_file = {}
        for doc_idx, cluster_id in enumerate(cluster_labels):
            if cluster_id not in self.inverted_file:
                self.inverted_file[cluster_id] = []
            self.inverted_file[cluster_id].append(doc_idx)
        
        # Step 3: Apply Product Quantization to each cluster
        self.cluster_pq = {}
        for cluster_id, doc_indices in self.inverted_file.items():
            if len(doc_indices) > m:  # Only apply PQ if cluster is large enough
                cluster_embeddings = embeddings[doc_indices]
                try:
                    pq = ProductQuantization(cluster_embeddings, m=m, k=min(k, len(doc_indices)//2))
                    self.cluster_pq[cluster_id] = pq
                except:
                    # Fallback for small clusters
                    self.cluster_pq[cluster_id] = None
        
        build_time = time.time() - start_time
        print(f"IVF+PQ index built in {build_time:.2f} seconds")
        
        # Calculate compression statistics
        pq_clusters = sum(1 for pq in self.cluster_pq.values() if pq is not None)
        print(f"Applied PQ to {pq_clusters}/{len(self.inverted_file)} clusters")
    
    def search(self, query, k=10, n_probes=4):
        """
        Search using combined IVF+PQ.
        """
        # Step 1: Find closest clusters (same as IVF)
        cluster_distances = []
        for cluster_id, centroid in enumerate(self.kmeans.cluster_centers_):
            distance = cosine_similarity([query], [centroid])[0][0]
            cluster_distances.append((cluster_id, distance))
        
        cluster_distances.sort(key=lambda x: x[1], reverse=True)
        top_clusters = [cluster_id for cluster_id, _ in cluster_distances[:n_probes]]
        
        # Step 2: Search in selected clusters using PQ where available
        all_candidates = []
        
        for cluster_id in top_clusters:
            if cluster_id not in self.inverted_file:
                continue
                
            doc_indices = self.inverted_file[cluster_id]
            
            if cluster_id in self.cluster_pq and self.cluster_pq[cluster_id] is not None:
                # Use PQ for this cluster
                pq = self.cluster_pq[cluster_id]
                local_indices, similarities = pq.search(query, k=len(doc_indices))
                # Convert local indices back to global indices
                global_indices = [doc_indices[i] for i in local_indices]
                candidates = list(zip(global_indices, similarities))
            else:
                # Use exact search for small clusters
                cluster_embeddings = self.embeddings[doc_indices]
                similarities = cosine_similarity([query], cluster_embeddings)[0]
                candidates = list(zip(doc_indices, similarities))
            
            all_candidates.extend(candidates)
        
        # Step 3: Combine and rank all candidates
        if not all_candidates:
            return np.array([]), np.array([])
        
        all_candidates.sort(key=lambda x: x[1], reverse=True)
        
        result_indices = [c[0] for c in all_candidates[:k]]
        result_similarities = [c[1] for c in all_candidates[:k]]
        
        return np.array(result_indices), np.array(result_similarities)

# Test combined IVF+PQ
print("Testing Combined IVF+PQ...")

try:
    ivf_pq_index = IVF_PQ(embeddings, n_clusters=50, m=8, k=256)
    
    start_time = time.time()
    ivf_pq_indices, ivf_pq_similarities = ivf_pq_index.search(query_vector, k=10, n_probes=4)
    ivf_pq_search_time = time.time() - start_time
    
    # Calculate recall
    recall = len(set(ivf_pq_indices[:10]) & set(brute_force_results['indices'][:10])) / 10
    
    print(f"\nIVF+PQ Results:")
    print(f"- Search time: {ivf_pq_search_time:.6f} seconds")
    print(f"- Speedup vs brute force: {search_time/ivf_pq_search_time:.1f}x")
    print(f"- Recall@10: {recall:.1%}")
    print(f"\nBest of both worlds: Fast search (IVF) + Memory efficiency (PQ)")
    
except Exception as e:
    print(f"IVF+PQ failed: {e}")
    print("This is normal - combining methods can be complex and may fail with small datasets")

## 8. Real-World Considerations and Next Steps

### What You've Learned

1. **Brute Force**: Perfect accuracy but O(n·d) complexity - only for small datasets
2. **HNSW**: Excellent speed/accuracy balance with O(log n) search time
3. **IVF**: Good for medium datasets with tunable accuracy via n_probes
4. **Product Quantization**: Massive memory savings with controllable accuracy loss

### Production Considerations

When building real vector search systems, consider:

1. **Dataset Size**: 
   - < 10K vectors: Brute force is fine
   - 10K - 1M vectors: HNSW or IVF
   - > 1M vectors: IVF + PQ or specialized databases

2. **Memory Constraints**:
   - Unlimited memory: HNSW for best performance
   - Limited memory: Product Quantization
   - Balanced: IVF with moderate n_probes

3. **Update Frequency**:
   - Static data: Any method works
   - Frequent updates: Avoid methods that require full rebuilds

4. **Latency Requirements**:
   - Real-time: HNSW with optimized parameters
   - Batch processing: Any method is suitable

### Next Steps

To continue learning:

1. **Try with real embeddings**: Use sentence transformers or OpenAI embeddings
2. **Scale up**: Test with larger datasets (100K+ vectors)
3. **Production libraries**: Explore Faiss, Annoy, or Hnswlib
4. **Vector databases**: Try Pinecone, Weaviate, or Qdrant
5. **Hybrid search**: Combine vector search with text search

### Additional Resources

- [Faiss Documentation](https://github.com/facebookresearch/faiss)
- [HNSWlib](https://github.com/nmslib/hnswlib)
- [Vector Database Comparison](https://github.com/erikbern/ann-benchmarks)
- [OpenSearch k-NN Plugin](https://opensearch.org/docs/latest/search-plugins/knn/)


In [ ]:
# Final summary
print("🎉 Congratulations!")
print("\nYou've successfully implemented and compared 4 different vector search methods:")
print("1. ✅ Brute Force Search (O(n·d) complexity)")
print("2. ✅ HNSW Graph Search (O(log n) complexity)")
print("3. ✅ IVF Clustering (O(n/k) complexity)")
print("4. ✅ Product Quantization (Massive memory compression)")
print("\nYou now understand the mathematical foundations and practical trade-offs")
print("of vector search methods used in production systems!")
print("\n🚀 Ready to build your own vector search applications!")